# Mục lục
1. Mô hình R-CNN
2. Selective search
3. Bouding Box Regression
4. IoU

# Tài liệu tham khảo
- https://lilianweng.github.io/lil-log/2017/12/31/object-recognition-for-dummies-part-3.html#r-cnn
- https://github.com/yangxue0827/RCNN
- https://github.com/Liu-Yicheng/R-CNN
- https://helpex.vn/article/tu-r-cnn-den-r-cnn-nhanh-hon-su-phat-trien-cua-cong-nghe-phat-hien-doi-tuong-5c66458aae03f601287658bf
- https://nttuan8.com/bai-11-object-detection-voi-faster-r-cnn/#RCNN_(Region_with_CNN_feature)
- Rich feature hirarchies for accurate object detection and sematic segmentaion (Paper)
- https://d2l.ai/chapter_computer-vision/rcnn.html
- https://thorpham.github.io/blog/
- http://cs.brown.edu/people/pfelzens/segment/

# 1. Mô hình R-CNN
Mạng CNN và các kiến trúc CNN hiện đại giúp phân loại nhận dạng hình ảnh, mạng CNN chỉ phân loại được ảnh chứa 1 đối tượng. Tuy nhiên, trong cuộc sống bình thường thì ảnh không chỉ chứa 1 đối tượng mà còn có thể chứa nhiều đối tượng. Nếu hình ảnh chứa nhiều đối tượng thì CNN chưa thực hiện được việc phân vùng đối tượng, các bài toán nhận dạng nhiều đối tượng trong ảnh gọi là `object detection`.

Về cơ bản, một bài toán object detection được chia thành 2 bài toán nhỏ sau đây:
* Xác định các bouding box (hình chữ nhật) quanh đối tượng
* Với mỗi bouding box cần phân loại xem đó là đối tượng gì.

R-CNN là một trong số những thuật toán cơ bản và là nền tảng cho nhiều thuật toán khác để nhận dạng nhiều đối tượng trong ảnh.

R-CNN (Girshick et al, 2014) là viết tắt của cụm từ `"Region-based Convolutional Neural Networks"`. Ý tưởng chính của R-CNN trải qua 2 bước:
1. Dùng thuật toán Selective Search để lấy ra khoảng 2000 bouding box trong ảnh mà có khả năng chứa đối tượng.
2. Với mỗi bouding box, ta xác định xem nó là đối tượng nào.

![RCNN](./images/RCNN.png)

Cách thức hoạt động của R-CNN có thể được tóm tắt như sau:
1. Huấn luyện (hoặc sử dụng pre-train) một mô hình CNN phân loại (ví dụ như AlexNet)
2. **Selective search** được sử dụng để tìm kiếm các region proposal trên hình ảnh đầu vào (khoảng 2000 vùng đề xuất trên mỗi ảnh). Các vùng này có thể có đối tượng hoặc không và chúng có kích thước khác nhau.
3. Các vùng đề xuất được **warped** để đưa về kích thước cố định mà mạng CNN yêu cầu.
4. Thực hiện fine-tuning (nếu sử dụng pre-train) mạng CNN cho K+1 lớp; trong đó K là số đối tượng chúng ta cần phân lớp và bổ sung thêm 1 lớp gọi là *background*.
5. Với mỗi vùng đề xuất, thực hiện lan truyền qua mạng CNN sẽ tạo ra một vector đặc trưng. Vector đặc trưng này sau đó sử dụng 1 **binary SVM** được đào tạo cho từng lớp 1 cách độc lập. (Xác định đối tượng có thuộc về mỗi loại hay không?)
6. Sử dụng **bouding box regression** để tinh chỉnh vị trí các vùng đề xuất. Đối với mỗi lớp, huấn luyện 1 mô hình hồi quy để xác định xem hộp có được tối ưu hoá hay không?

![r-cnn.svg](./images/r-cnn.svg)

# 2. Selective search
Thuật toán selective search đóng vai trò quan trọng trong việc đề xuất vùng dựa trên đầu vào là các phần phân đoạn. Đầu vào của thuật toán là ảnh màu, đầu ra là khoảng 2000 region proposal (bouding box) mà có khả năng chứa các đối tượng.

Đầu tiên ảnh được phân đoạn (segment) qua thuật toán `Graph based image segmentation`, thuật toán dựa trên đồ thị và không sử dụng deep learning.

![selective search](./images/graph-based-segmentation.png)

Rõ ràng, ta không thể dùng mỗi màu trong đầu ra để là một region proposal được vì:
* Mỗi đối tượng trong ảnh có thể chứa nhiều hơn 1 màu.
* Các đối tượng bị che mất một phần như cái đĩa dưới cái chén không thể xác định được như hình trên.

Vì vậy, cần nhóm các vùng màu với nhau để làm region proposal.

Từ đó,selective seach trải qua các bước như sau:
1. Thêm toàn bộ các khoanh vùng tương ứng đối với mỗi phần phân đoạn vào danh sách region proposal (Đề xuất vùng).
2. Tiến hành nhóm các phần phân đoạn dựa trên độ tương quan.
3. Quay lại bước 1 cho đến khi đạt só lượng region proposal cần thiết.

Việc đánh giá độ tương quan của các phần phân đoạn để nhóm thành 1 nhóm có thể dựa trên các tiêu chí: màu sắc, kết cấu, kích thước và hình dạng.

![selectivesearch](./images/selectivesearch.png)

# 3. Bounding box regression
Các tác giả sử dụng bouding box regression để cải thiện hiệu suất. Sau khi dự đoán cho mỗi vùng với 1 class riêng bằng SVM, các tác giả đưa thêm một tầng hồi quy để tinh chỉnh lại các bouding box dự đoán.

Đầu vào của thuật toán là một tập $N$ cặp $(p,g)$ với $p$ là box dự đoán và $g$ là box thuật, box dự đoán có toạ độ $p=(p_x, p_y, p_w, p_h)$ (Toạ độ trung tâm, chiều rộng, chiều cao), box thật $g=(g_x, g_y, g_w, g_h)$. Mục tiêu của chúng ta là cần học một chuyển đổi ánh xạ từ box dự đoán $p$ thành một box thật (ground-truth) $g$.

Chúng ta tham số hoá chuyển đổi trong 4 hàm: $d_x(p), d_y(p), d_w(p), d_h(p)$. Với mỗi đầu vào $p$, thực hiện chuyển đổi như sau:
$$\begin{aligned}
\hat{g}_x &= p_w d_x(\mathbf{p}) + p_x \\
\hat{g}_y &= p_h d_y(\mathbf{p}) + p_y \\
\hat{g}_w &= p_w \exp({d_w(\mathbf{p})}) \\
\hat{g}_h &= p_h \exp({d_h(\mathbf{p})})
\end{aligned}$$

![RCNN-bbox-regression.png](./images/RCNN-bbox-regression.png)

Một lợi ích rõ ràng của việc áp dụng chuyển đổi như vậy là tất cả các hàm hiệu chỉnh hộp giới hạn, $d_i(p)$ với $i \in \{x,y,w,h\}$, có thể lấy một vài giá trị giữa $[-\inf, \inf]$. Mục tiêu là học:
$$\begin{align}
t_x = (g_x-p_x)/p_w \\
t_y = (g_y-p_y)/p_h \\
t_w = log(g_w/p_w) \\
t_h = log(g_h/p_h) \\
\end{align}$$
Mục tiêu của chúng ta là cần cực tiểu hoá hàm mất mát MSE với regularization:
$$\mathcal{L}_\text{reg} = \sum_{i \in \{x, y, w, h\}} (t_i - d_i(\mathbf{p}))^2 + \lambda \|\mathbf{w}\|^2$$

Một điều đáng chú ý là không phải tất cả các box đề xuất đều có các ground-box tương ứng. Ở đây, chỉ có các box đề xuất mà có IoU với box thật ít nhất 0,6 trở lên mới được giữ lại để đào tạo mô hình.

# 4. IoU
IoU được sử dụng trong bài toán object detection, để đánh giá xem bounding box dự đoán đối tượng khớp với ground truth thật của đối tượng.

![IoU-2](images/IoU-2.png)

Nhận xét:
* Chỉ số IoU trong khoảng [0,1]
* IoU càng gần 1 thì bounding box dự đoán càng gần ground truth

![IoU_ex](./images/IoU_ex.png)